In [6]:
import pandas as pd
import matplotlib as mpl

# 用户信息和实验开始时间处理

In [1]:
user_d = {
    "name": "wuhui",
    "id": "02",
    "cond": "int",
    "s1":"2018-06-26 13:02:52.642453",
    "s2":"2018-06-26 13:02:56.792038",
    "s3":"2018-06-26 13:22:42.319474",
    "s4":"2018-06-26 13:22:44.419264",
}

In [2]:
tim1 = user_d["s1"].split(" ")[1].split(":")
tim2 = user_d["s2"].split(" ")[1].split(":")
tim3 = user_d["s3"].split(" ")[1].split(":")
tim4 = user_d["s4"].split(" ")[1].split(":")
START_T = ((float(tim1[0])*60*60 + float(tim1[1])*60 + float(tim1[2]))+(float(tim2[0])*60*60 + float(tim2[1])*60 + float(tim2[2])))/2
END_T = ((float(tim3[0])*60*60 + float(tim3[1])*60 + float(tim3[2]))+(float(tim4[0])*60*60 + float(tim4[1])*60 + float(tim4[2])))/2

In [7]:
print("="*40)
print("Confirm Input Data:\n")
for k in user_d:
    print("%s - %s\n"%(k,user_d[k]))
print("\nThe exp start at: ",START_T,"; End at: ",END_T,"\n")
print("="*40)

Confirm Input Data:

name - wuhui

id - 02

cond - int

s1 - 2018-06-26 13:02:52.642453

s2 - 2018-06-26 13:02:56.792038

s3 - 2018-06-26 13:22:42.319474

s4 - 2018-06-26 13:22:44.419264


The exp start at:  46974.717245499996 ; End at:  48163.369369 



# 导入传感器数据并换算

In [8]:
print("\nImport the Sensor Data:\n")
f_d = ["02_wuhui_201806261323"]
f = pd.read_table(f_d[0],sep=" ",header=None,names=["time","sensor1","sensor2","sensor3","sensor4","x","y","f"])
f.time = f.time/1000
sensor_start_local_time = f.iloc[0].time
f["real_time"] = f.time - sensor_start_local_time + START_T


Import the Sensor Data:



In [9]:
of = f.reindex(columns="real_time sensor1 sensor2 sensor3 sensor4 x y f".split(" ")).rename(columns={"real_time":"time"})
print("\nThis is the Sensor clear data:\n")
print(of.head(),"\n")


This is the Sensor clear data:

           time   sensor1   sensor2   sensor3   sensor4    x    y         f
0  46974.717245  0.000000  0.072165  0.010366  0.011565  0.0  0.0  0.094095
1  46974.721245  0.000000  0.000000  0.000000  0.069388  0.0  0.0  0.069388
2  46974.732245  0.000000  0.000000  0.000000  0.069388  0.0  0.0  0.069388
3  46974.738245  0.009918  0.082474  0.000000  0.046259  0.0  0.0  0.138651
4  46974.752245  0.009918  0.082474  0.000000  0.046259  0.0  0.0  0.138651 



In [10]:
of.tail()

,time,sensor1,sensor2,sensor3,sensor4,x,y,f
89230,48160.379245,11.1085,12.4021,17.6870,16.1211,-0.118782,2.69480,57.3186
89231,48160.402245,11.0788,12.5773,17.7793,15.9939,-0.124857,2.64249,57.4292
89232,48160.409245,11.1680,12.6598,17.6972,16.0054,-0.086917,2.57468,57.5305
89233,48160.436245,11.1680,12.6907,17.6460,15.9476,-0.076433,2.54163,57.4523
89234,48160.442245,11.1482,12.7320,17.6460,15.8435,-0.095281,2.51248,57.3696


# 导入实验数据并进行时间换算

s 表格是 matlab 计算的数据，我们需要将这些从计算机开机的时间转换成我们定义的时间，a1点的时间为a1点出现时计算机开机时间，start_secs为a1点的日期，经过换算后对应的我们定义的时间（当天的秒数）。

In [18]:
print("\nDealing with MATLAB Data\n")
s_d = ["02_wuhui_int.xlsx"]
s = pd.read_excel(s_d[0],header=None,
                  names=["year","month","day","hour","mint","secs","a1","a2","b1","b2","c1","c2","d1","d2","e1","e2"])
s["start_secs"] = s["hour"]*60*60 + s["mint"]*60 + s["secs"]
s = s.reindex(columns=["start_secs","a1","a2","b1","b2","c1","c2","d1","d2","e1","e2"])
print("The file looks like:\n",s.head(3))


Dealing with MATLAB Data

The file looks like:
    start_secs           a1           a2           b1           b2  \
0   47203.648  2938.850332  2940.066931  2940.100248  2941.100162   
1   47208.624  2943.833262  2945.049893  2945.066538  2946.066469   
2   47213.839  2949.049500  2950.266144  2950.282830  2951.282786   

            c1           c2           d1           d2           e1  \
0  2941.116824  2941.533450  2941.550146  2942.550038  2942.642776   
1  2946.083131  2946.499769  2946.516437  2947.916340  2947.933301   
2  2951.299425  2951.916041  2951.932693  2953.532594  2953.549727   

            e2  
0  2943.816636  
1  2949.032931  
2  2954.482525  


In [19]:
os = pd.DataFrame()
for col in list("a1 a2 b1 b2 c1 c2 d1 d2 e1 e2".split(" ")):
    
    os[col] = s[col] - s.a1 + s.start_secs

os 表格表示的就是所有使用我们计时单位（当天的秒数）的时间

In [20]:
print("Clearning Data...\n")
print("The Clear Data like this \n",os.head())

Clearning Data...

The Clear Data like this 
           a1            a2            b1            b2            c1  \
0  47203.648  47204.864599  47204.897916  47205.897830  47205.914493   
1  47208.624  47209.840631  47209.857276  47210.857207  47210.873869   
2  47213.839  47215.055644  47215.072329  47216.072285  47216.088925   
3  47219.288  47220.404567  47220.421259  47220.821215  47220.837889   
4  47223.708  47225.024548  47225.041225  47226.041158  47226.057824   

             c2            d1            d2            e1            e2  
0  47206.331118  47206.347814  47207.347707  47207.440444  47208.614304  
1  47211.290507  47211.307175  47212.707078  47212.724039  47213.823669  
2  47216.705541  47216.722193  47218.322094  47218.339226  47219.272025  
3  47221.354515  47221.371180  47222.371116  47222.388148  47223.671028  
4  47226.474487  47226.491132  47227.691035  47227.708654  47228.974282  


# 数据对齐

In [44]:
print("="*40,"\nNow Dealing with Data Marge...\n")
oos = os.copy()
for current_alp in "a1 a2 b1 b2 c1 c2 d1 d2 e1 e2".split(" "):
    for current_lab in "similar_time sensor1 sensor2 sensor3 sensor4 x y f".split(" "):
        oos[current_alp+"_"+current_lab] = 0.00;
#对于每一种条件进行遍历
for current_alp in "a1 a2 b1 b2 c1 c2 d1 d2 e1 e2".split(" "):
    print("[PROCESSING] Now is %s , you may waiting for a long time...\n"%current_alp)
    #在每种条件的基础上，对每行数据进行遍历
    for r in os.index:
        #print("row now is %s\n"%r)
        os_time = os[current_alp][r]
        of_row = -1;
        of_row_temp = -1;
        #对于os的每一个单元格，遍历of查找最精确的数据
        for m in of.index:
            of_time = of["time"][m]
            if abs(of_time - os_time) < 0.1:
                of_row_temp = m
            if abs(of_time - os_time) < 0.01:
                of_row = m
                break;
        if of_row == -1:
            if of_row_temp == -1:
                raise ValueError("没有匹配数据")
            else:
                of_row = of_row_temp
        #print("find the of_row is %s\n"%of_row)
        for current_lab in "sensor1 sensor2 sensor3 sensor4 x y f".split(" "):
            oos[current_alp+"_"+current_lab][r] = of[current_lab][of_row]
        oos[current_alp+"_similar_time"][r] = of["time"][of_row]
    #break;#DEBUG

now is a1 , you may waiting for a long time...

now is a2 , you may waiting for a long time...

now is b1 , you may waiting for a long time...

now is b2 , you may waiting for a long time...

now is c1 , you may waiting for a long time...

now is c2 , you may waiting for a long time...

now is d1 , you may waiting for a long time...

now is d2 , you may waiting for a long time...

now is e1 , you may waiting for a long time...

now is e2 , you may waiting for a long time...



In [60]:
print("\nThe data looks like this:\n",oos.head(20))


The data looks like this:
            a1            a2            b1            b2            c1  \
0   61707.113  61708.629597  61708.646263  61709.646196  61709.662833   
1   61712.494  61713.610600  61713.627242  61714.627178  61714.643831   
2   61716.826  61717.942610  61717.959242  61718.959204  61718.975828   
3   61720.680  61722.196587  61722.213234  61723.213124  61723.229828   
4   61725.193  61726.509597  61726.526274  61727.526207  61727.542849   
5   61730.092  61731.108634  61731.125264  61732.725157  61732.741792   
6   61734.825  61736.341603  61736.358222  61737.358167  61737.374792   
7   61739.310  61740.526610  61740.543273  61742.143048  61742.159812   
8   61744.425  61745.841639  61745.858311  61746.258257  61746.274945   
9   61748.958  61750.274597  61750.291244  61751.291204  61751.307834   
10  61753.508  61754.524627  61754.541279  61755.541192  61755.557851   
11  61758.057  61759.073651  61759.090326  61760.090258  61760.106894   
12  61761.990  61763.20

In [48]:
start_time = user_d["s1"].strip().replace(" ","_").replace(".","_").replace("-","_").replace(":","_")
oos.to_csv("%s_%s_%s_clear_data_%s.csv"%(user_d["name"],user_d["id"],user_d["cond"],start_time),encoding="utf8")
print("\nFinished. Output to %s_%s_%s_clear_data_%s.csv\n\n"%(user_d["name"],user_d["id"],user_d["cond"],start_time))
print("="*40+"\n")
input()